In [1]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
using JPEC, Plots

  Activating project at `~/Projects/Perturbed-Equilibrium/JPEC.worktrees/vacuum_julia`
Precompiling project...
   4615.6 ms  ✓ JPEC
  1 dependency successfully precompiled in 6 seconds. 181 already precompiled.
  1 dependency had output during precompilation:
┌ JPEC
│  gfortran -fPIC -frecursive -c defs.f -o defs.o
│  gfortran -fPIC -frecursive -c spline.f -o spline.o
│  gfortran -fPIC -frecursive -c cspline.f -o cspline.o
│  gfortran -fPIC -frecursive -c bicube.f -o bicube.o
│  gfortran -fPIC -frecursive -c spline_c_api.f -o spline_c_api.o
│  gfortran -shared -o libspline.dylib defs.o spline.o cspline.o bicube.o spline_c_api.o -framework Accelerate
│  mv libspline.dylib ../../../deps/
│  [ Info: Splines-fortran compiled well
│  gfortran -fPIC -finit-local-zero -frecursive -std=legacy -c vacuum_global.f -o vacuum_global.o
│  cc    -c -o bzio.o bzio.c
│  gfortran -fPIC -finit-local-zero -frecursive -std=legacy -c aleg_new.f -o aleg_new.o
│  gfortran -fPIC -finit-local-zero -frecursive -

In [10]:
function fouran!(gij::Matrix{Float64}, gil::Matrix{Float64},
                 cs::Matrix{Float64}, m00::Int, l00::Int,
                 lmin::Vector{Int}, lmax::Vector{Int}, mth::Int)

    jmax1 = lmax[1] - lmin[1] + 1

    # Zero out relevant gil block
    for l1 in 1:jmax1
        for i in 1:mth
            gil[m00 + i, l00 + l1] = 0.0
        end
    end

    # Accumulate
    for l1 in 1:jmax1
        for j in 1:mth
            for i in 1:mth
                gil[m00 + i, l00 + l1] += cs[j, l1] * gij[i, j]
            end
        end
    end

    return nothing
end

fouran! (generic function with 1 method)

In [19]:
mth   = 4
lmin  = [1]
lmax  = [3]
m00   = 0
l00   = 0

gij = rand(mth, mth)
cs  = rand(mth, lmax[1]-lmin[1]+1)
gil = zeros(mth+5, lmax[1]-lmin[1]+1+5)

fouran!(gij, gil, cs, m00, l00, lmin, lmax, mth)

In [20]:
println("gil = ")
println(gil)              # prints entire matrix

gil = 
[0.8521479782441241 0.7207050569791131 0.6127998030665539 0.0 0.0 0.0 0.0 0.0; 1.9837799162945684 1.2406360642748808 0.9566077327138862 0.0 0.0 0.0 0.0 0.0; 1.4202980227048911 1.0275087724622787 0.7685186420562836 0.0 0.0 0.0 0.0 0.0; 0.9446675824743859 0.935989103055863 0.6825729593626579 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]


In [17]:
function foranv!(gil::Array{Float64,2}, gll::Array{Float64,2}, cs::Array{Float64,2},
                 m00::Int, l00::Int, mth::Int, lmin::Int, lmax::Int,
                 dth::Float64)

    jmax1 = lmax - lmin + 1

    # Zero out the block of gll that will be filled
    for l1 in 1:jmax1
        for l2 in 1:jmax1
            gll[l2,l1] = 0.0
        end
    end

    # Main summation
    for l1 in 1:jmax1
        for l2 in 1:jmax1
            for i in 1:mth
                gll[l2,l1] += dth * cs[i,l2] * gil[m00+i, l00+l1] * 2π
            end
        end
    end

    return gll
end

foranv! (generic function with 2 methods)

In [18]:
function test_foranv()
    # Example parameters (adjust to match your case)
    mth   = 4        # number of poloidal points
    lmin  = 1
    lmax  = 3
    m00   = 0
    l00   = 0
    dth   = 0.1

    nths2, nfm2 = 8, 8  # dimensions for gil
    nfm         = 4     # dimension for cs and gll

    # Dummy input data
    gil = randn(nths2, nfm2)
    cs  = randn(mth, nfm)
    gll = zeros(nfm, nfm)

    # Run the translation
    result = foranv!(gil, gll, cs, m00, l00, mth, lmin, lmax, dth)

    println("gil =")
    println(gil)
    println("\ncs =")
    println(cs)
    println("\ngll result =")
    println(result)

    return result
end

# Run test
test_foranv()


gil =
[0.9689672392890408 0.9121767008774335 0.09856309894024685 -0.7162631237574117 -0.3191565025919801 1.3922171529885254 -0.1256348671099153 0.6275632040148752; 0.7772682899932158 0.7174260497145164 0.4339418920704299 1.6457444805739103 -0.22547799771931396 -0.4873835583830081 -1.0804410429171751 -1.1545538277945562; -0.6058275070385057 -0.8575971748105143 -0.2001402411718884 -1.7946304268873379 0.5217103318242219 0.9258561305161837 0.471636948874507 -1.8829855354869278; -0.14591032292568135 -0.30068622900827113 0.10843884742819349 -0.39888903119790436 -0.8310931447358131 -0.24955451631242462 -0.01147022190718461 0.6699081021584657; 1.4590735953920912 -0.8132024654348188 0.3107257577196023 0.28997927692216435 1.4638767736245855 0.876520376929424 1.2562190354012197 -0.8389054853686562; 1.1437859459856663 -0.6201704601707787 -0.4575163605576846 -0.5943593511755471 0.5019110082943155 1.6720330503888658 1.0477218699712174 0.32703636872068714; 0.9091924061377582 -1.0229483643752972 0.447

4×4 Matrix{Float64}:
  0.225037   0.121759   0.161266  0.0
 -1.02561   -0.94709   -0.127691  0.0
  2.10207    2.07597    0.269494  0.0
  0.0        0.0        0.0       0.0